<a href="https://colab.research.google.com/github/Mohdd-Afaan/data-science-master-2.0/blob/main/Bokeh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1. How can you create a Bokeh plot using Python code?

Ans: To create a Bokeh plot using Python code, you typically follow these steps:
1) Install Bokeh: Make sure you have Bokeh installed.
2) Import Bokeh: Import the necessary modules from Bokeh.
3) Prepare Data: Create or load the data you want to visualize.
4) Create a Figure: Initialize a Bokeh figure.
5) Add Glyphs: Add glyphs (markers or lines) to the figure to represent your  
   data.
6) Show the Plot: Display the plot.

Q2. What are glyphs in Bokeh, and how can you add them to a Bokeh plot? Explain with an example.

Ans:
In Bokeh, glyphs are visual elements that represent data on a plot. Glyphs are the building blocks of Bokeh visualizations and can include markers, lines, bars, and other shapes. Adding glyphs to a Bokeh plot involves using specific glyph functions provided by Bokeh to create the desired visual representation of your data.

In [5]:
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()
from bokeh.plotting import figure , output_file, show

In [6]:
from bokeh.sampledata.iris import flowers

In [9]:
output_file("example.html")
p= figure(title ='Flower')
p.xaxis.axis_label = 'sepal_length'
p.yaxis.axis_label = 'sepal_width'
p.circle(flowers['sepal_length'],flowers['sepal_width'])
show(p)

Q3. How can you customize the appearance of a Bokeh plot, including the axes, title, and legend?

Ans: In Bokeh, you can customize the appearance of a plot by adjusting various attributes such as axes, title, legend, colors, and more.

In [10]:
from bokeh.models import Legend

In [14]:
output_file("example.html")
p= figure(title ='Flower')
p.xaxis.axis_label = 'sepal_length'
p.yaxis.axis_label = 'sepal_width'
p.circle(flowers['sepal_length'],flowers['sepal_width'],fill_color= 'black' ,legend_label = 'this is my data point')
show(p)

Q4. What is a Bokeh server, and how can you use it to create interactive plots that can be updated in
real time?

Ans: Bokeh Server is a component of the Bokeh library that allows you to create interactive, real-time, and data-driven web applications. With Bokeh Server, you can build dynamic dashboards and applications where the plots update in response to user interactions or changes in the underlying data. To create interactive plots that can be updated in real-time using Bokeh, you can use Bokeh Server. Bokeh Server allows you to build dynamic web applications with interactive and live updates. Here are the required steps:
Step 1: Install Bokeh: Make sure you have Bokeh installed.
Step 2: Create a Bokeh Server Script: Create a Python script (e.g., app.py)
Step 3: Run the Bokeh Server: Open a terminal, navigate to the directory containing your app.py script, and run


here is an example

In [1]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.io import curdoc
from bokeh.plotting import figure
import requests
import time
from datetime import datetime


data_source =ColumnDataSource(data= {'Close':[], 'DateTime':[]})


fig = figure(x_axis_type='datetime',
             width= 950, height= 450,
             tooltips= [('Close','@Close')], title = 'Bitcoin Close Price Live (Every Second)')

fig.line(x='DateTime', y='Close', line_color='tomato', line_width= 3.0, source= data_source)

fig.xaxis.axis_label= 'Date'
fig.yaxis.axis_label= 'Price ($)'


def update():
    global data_source
    response = requests.get('https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD')
    data = response.json()
    new_row = {'Close':[data['USD']],'DateTime':[datetime.now()]}
    data_source.stream(new_row)


doc = curdoc()

doc.add_periodic_callback(update,1000)

curdoc().add_root(fig)

Q5. How can you embed a Bokeh plot into a web page or dashboard using Flask or Django?

Ans: Embedding a Bokeh plot into a web page or dashboard using Flask involves creating a Bokeh plot, rendering it to HTML, and then integrating it into your web framework. Here are instructions for Flask:
1) Install Flask
2) Create a Flask App: Create a file (e.g., app.py)
3) Create HTML Template: Create a folder named templates in the same directory
   as your Flask app. Inside the templates folder, create an HTML file (e.g., index.html)
4) Run Flask App

In [ ]:
from flask import Flask, render_template
from bokeh.embed import server_document
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import curdoc
import requests
from datetime import datetime

app = Flask(__name__)

@app.route('/')
def index():
    # Use server_document to embed Bokeh app into the HTML template
    script = server_document('http://localhost:5050/')
    return render_template('index.html', script=script, template="Flask")

def modify_doc(doc):
    data_source = ColumnDataSource(data={'Close': [], 'DateTime': []})

    fig = figure(x_axis_type='datetime',
                 width=950, height=450,
                 tooltips=[('Close', '@Close')], title='Bitcoin Close Price Live (Every Second)')

    fig.line(x='DateTime', y='Close', line_color='tomato', line_width=3.0, source=data_source)

    fig.xaxis.axis_label = 'Date'
    fig.yaxis.axis_label = 'Price ($)'

    def update():
        response = requests.get('https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD')
        data = response.json()
        new_row = {'Close': [data['USD']], 'DateTime': [datetime.now()]}
        data_source.stream(new_row)

    doc.add_periodic_callback(update, 1000)
    doc.add_root(fig)

# Note: Bokeh server application setup
apps = {'/': Application(FunctionHandler(modify_doc))}
server = Server(apps, port=5050, allow_websocket_origin=["localhost:5050"])
server.start()

if __name__ == '__main__':
    print('Opening Flask app with embedded Bokeh plot on http://localhost:5000/')
    server.io_loop.add_callback(server.show, "/")
    server.io_loop.start()